In [1]:
import numpy as np
import xarray as xr
import pandas as pd
# import ipyvolume as ipv
# import pyvista as pv
# import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

dir0 = '/Users/kugelblitz/Documents/USP/models/Slab2.0/Slab2Distribute_Mar2018/Slab2_TXT/'

In [2]:
#phi_slab2_dip_02.26.18.xyz

################################################################################
# Read Slab2 file data -> top surface of the slab
################################################################################
data = pd.read_csv('./src/sam_slab2_dep_02.23.18.xyz', names=['lon', 'lat', 'dep'])
dep = data.pivot_table(index='lon', columns='lat', values='dep', dropna=False).T.values
dep = xr.DataArray(1.0E3 * dep, coords=[np.unique(data['lat']), np.unique(data['lon'])], dims=['lat', 'lon'])
dep.attrs['units'] = 'm'

################################################################################
# Read Slab2 file data -> thickness of the slab
################################################################################
data = pd.read_csv('./src/sam_slab2_thk_02.23.18.xyz', names=['lon', 'lat', 'thk'])
thk = data.pivot_table(index='lon', columns='lat', values='thk', dropna=False).T.values
thk = xr.DataArray(1.0E3 * thk, coords=[np.unique(data['lat']), np.unique(data['lon'])], dims=['lat', 'lon'])
thk.attrs['units'] = 'm'

# ################################################################################
# # Read Slab2 file data -> strike angle of the slab
# ################################################################################
data = pd.read_csv('./src/sam_slab2_str_02.23.18.xyz', names=['lon', 'lat', 'str'])
str = data.pivot_table(index='lon', columns='lat', values='str', dropna=False).T.values
str *= np.pi / 180.
str = xr.DataArray(str, coords=[np.unique(data['lat']), np.unique(data['lon'])], dims=['lat', 'lon'])
str.attrs['units'] = 'rad'

################################################################################
# Read Slab2 file data -> dip angle of the slab
################################################################################
data = pd.read_csv('./src/sam_slab2_dip_02.23.18.xyz', names=['lon', 'lat', 'dip'])
dip = data.pivot_table(index='lon', columns='lat', values='dip', dropna=False).T.values
dip *= np.pi / 180.
dip = xr.DataArray(dip, coords=[np.unique(data['lat']), np.unique(data['lon'])], dims=['lat', 'lon'])
dip.attrs['units'] = 'rad'

In [ ]:
X = dep.lon
Y = dep.lat
X, Y = np.meshgrid(X, Y)
Z = - dep.values / 1_000.0
Z = ~np.isnan(Z)

fig = plt.figure(figsize=plt.figaspect(0.5))

ax = fig.add_subplot(1, 1, 1, projection='3d')

surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
ax.set_zlim(np.min(Z), np.max(Z))
fig.colorbar(surf, shrink=0.5, aspect=10)

plt.show()

In [18]:
# cond = ((str > 0.0) & (str < np.pi / 2.)) | ((str > 3. * np.pi / 2.) & (str < 2 * np.pi))
# dip.values[cond] *= (-1)

# btm1 = dep.values - thk.values
x1 = dep['lon'].values * 111000.
y1 = dep['lat'].values * 111000.
# x1, y1 = np.meshgrid(x1, y1)

# dLonCos = thk.values * np.cos(dip.values)
# btm2 = dep.values - np.nan_to_num(dLonCos)

dLonSin = thk.values * np.sin(dip.values)
dLatSin = thk.values * np.sin(str.values)
#dLatCos = thk.values * np.cos(str.values)
x2 = x1 + (np.nan_to_num(dLonSin)) #- (np.nan_to_num(dLatCos))
y2 = y1 + (np.nan_to_num(dLatSin))


grid_dep = pv.StructuredGrid(x1, y1, dep.values)
grid_btm = pv.StructuredGrid(x2, y2, btm2)
#grid_btm2 = pv.StructuredGrid(xx,y,btm2.values)
###grid_thk = pv.StructuredGrid(x,y,btm.values)
##
#grid = pv.StructuredGrid(xx, y, btm2.values)
plotter = pv.Plotter()
plotter.add_mesh(grid_dep, color='grey', jupyter_backend='pythreejs')
plotter.add_mesh(grid_btm, color='blue', jupyter_backend='pythreejs')
#plotter.add_mesh(grid_btm, color='green')
plotter.show()


# #yy = dep['lat'].values * 111.
# #xxx, yy = np.meshgrid(dep['lon'].values * 111., yy)
# #lat_mesh = xr.DataArray(x)

# #slab2 = xr.Dataset({'dep': dep, 'thk': thk, 'dip': dip, 'str': str, 'btm': btm})
# #
# #
# #
# ##df = xr.DataArray(dep)
# ##slab2 = xr.Dataset(df)
# #
# #
# #
# #cpos = [(38850., -9100., 6660.),
# #		(32000., -1900., -330.),
# #		(0., 0., 1.)]
# #
# #
# grid_dep = pv.StructuredGrid(x1, y1, dep.values)
# grid_btm = pv.StructuredGrid(x2, y2, btm2)
# #grid_btm2 = pv.StructuredGrid(xx,y,btm2.values)
# ###grid_thk = pv.StructuredGrid(x,y,btm.values)
# ##
# #grid = pv.StructuredGrid(xx, y, btm2.values)
# plotter = pv.Plotter()
# plotter.add_mesh(grid_dep, color='grey', jupyter_backend='pythreejs')
# plotter.add_mesh(grid_btm, color='blue', jupyter_backend='pythreejs')
# #plotter.add_mesh(grid_btm, color='green')
# plotter.show()


# # Plotting specific latitude
# #index = 700
# #plt.plot(x1 / 1.0E3, dep.values[index] / 1.0E3, label='dep')
# #plt.plot(x1 / 1.0E3, btm1[index] / 1.0E3, label='btm1', color=(0.95, 0.95, 0.95))
# #plt.plot(x2[index] / 1.0E3, btm2[index] / 1.0E3, label='btm2')
# #plt.legend()
# #plt.show()

ValueError: operands could not be broadcast together with shapes (1281,) (1281,581) 

In [9]:
x = dep.lon
y = dep.lat
x, y = np.meshgrid(x, y)
z = - dep.values / 1_000.0

fig = ipv.figure()
surf = ipv.plot_surface(x, y, z)
ipv.xlim(270, 310)
ipv.zlim(4000, 0)
ipv.show()

In [ ]:
np.floor(np.min(x)/1.0)

In [ ]:
# Make data
x = dep.lon
y = dep.lat
x, y = np.meshgrid(x, y)
z = dep.values/100000.0

# Create and plot structured grid
grid = pv.StructuredGrid(x, y, z)
# grid.threshold([0,1])
outline = grid.outline()
# threshold = grid.threshold([0,1])
# grid = grid.threshold()
# grid.plot(jupyter_backend='ipygany', show_scalar_bar=True)
# grid.plot(color='white', jupyter_backend='ipygany')
# grid.show()
# grid.plot(outline, color="k")


# plotter = pv.Plotter()
# plotter.add_mesh(outline, color='black')
# plotter.add_mesh(grid, color='white')
# plotter.show()

# from itkwidgets import view


# view(geometries=grid, geometry_colors='w')

In [ ]:
# from pyvista import demos

# # basic glyphs demo
# mesh = grid_dep

# # text = demos.logo.text_3d("I'm interactive!", depth=0.2)
# # text.points *= 0.1
# # text.translate([0, 1.4, 1.5])
# # mesh += text
# # mesh['Example Scalars'] = mesh.points[:, 0]

# mesh.plot(cpos='xy', jupyter_backend='ipygany', show_scalar_bar=True)

In [ ]:
outline